In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
import matplotlib.pyplot as plt
from matplotlib import pyplot
from numpy import std
import seaborn as sns
from scipy import stats
import json
import pandas as pd
import networkx as nx

In [2]:
df = pd.read_csv('philly_users_businesses_stars.csv')
df

,user_id,business_id,stars
0,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5.0
1,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5.0
2,eUta8W_HdHMXPzLBBZhL1A,04UD14gamNjLY0IDYVhHJg,1.0
3,smOvOajNG0lS4Pq7d8g4JQ,RZtGWDLCAtuipwaZ-UfjmQ,4.0
4,Dd1jQj7S-BFGqRbApFzCFw,YtSqYv1Q_pOltsVPSx54SA,5.0
...,...,...,...
967547,5d32aJe0a3_fSl3LIVYkWQ,EmrOCT7fArKI0JI_XF6aaw,1.0
967548,am7-gkH_PDz598oTdYSD6A,3gVSrS4kffGGZT8oXHsIcw,3.0
967549,-AkziDwQ8hv2COTDBBUpig,aunmz06iWvo3bd6MMHEbqg,3.0
967550,7ziWZULyiZv2TesYNMFf4g,qQO7ErS_RAN4Vs1uX0L55Q,4.0


                                            -Step 1-

In [3]:
B_list = (df['business_id'].value_counts()[df['business_id'].value_counts() >= 50].index).tolist()
B_list[0]

'ytynqOUb3hjKeJfRj5Tshw'

In [4]:
def prune_dataframe(df, B_list, min_user_reviews, min_business_reviews):
    # filter out users with less than min_user_reviews reviews
    user_review_counts = df.groupby('user_id').size()
    df_filtered = df[df['user_id'].isin(user_review_counts[user_review_counts >= min_user_reviews].index)]
    
    # filter out businesses that are not in B or have less than min_business_reviews reviews
    business_review_counts = df_filtered.groupby('business_id').size()
    df_filtered = df_filtered[df_filtered['business_id'].isin(B_list)]
    
    # filter out users that have not reviewed the remaining businesses or have less than min_user_reviews reviews on them
    user_review_counts = df_filtered.groupby('user_id').size()
    df_filtered = df_filtered[df_filtered['user_id'].isin(user_review_counts[user_review_counts >= min_user_reviews].index)]
    business_review_counts = df_filtered.groupby('business_id').size()
    df_filtered = df_filtered[df_filtered['business_id'].isin(business_review_counts[business_review_counts >= min_business_reviews].index)]
    
    # get the sums U and B
    U = df_filtered['user_id'].unique()
    B = df_filtered['business_id'].unique()
    
    return df_filtered, U, B

In [5]:
df_pruned , prev_U , prev_B = prune_dataframe(df, B_list, 30, 50)
df_pruned , U , B = prune_dataframe(df_pruned, B_list, 30, 50)
len(prev_U)

3058

In [6]:
min_user_reviews = 30
min_business_reviews = 50

while True:
    df_pruned, U, B = prune_dataframe(df_pruned, B, min_user_reviews, min_business_reviews)
    print(len(U))
    if U.all() == prev_U.all() and B.all() == prev_B.all():
        
        # Stop if the sums U and B haven't changed since the previous iteration
        break
    prev_U, prev_B = U, B
while True:
    df_pruned, U, B = prune_dataframe(df_pruned, B, min_user_reviews, min_business_reviews)
    print(len(U))
    if U.all() == prev_U.all() and B.all() == prev_B.all():
        
        # Stop if the sums U and B haven't changed since the previous iteration
        break
    prev_U, prev_B = U, B

1819
1776
1757
1749


In [7]:
len(U)

1749

In [8]:
df_pruned

,user_id,business_id,stars
3,smOvOajNG0lS4Pq7d8g4JQ,RZtGWDLCAtuipwaZ-UfjmQ,4.0
5,IQsF3Rc6IgCzjVV9DE8KXg,eFvzHawVJofxSnD7TgbZtg,5.0
10,ZGjgfSvjQK886kiTzLwfLQ,EtKSTHV5Qx_Q7Aur9o4kQQ,5.0
11,IKbjLnfBQtEyVzEu8CuOLg,VJEzpfLs_Jnzgqh5A_FVTg,4.0
13,NUtIAX-ygn474tDg5nmesg,6LCZLGa09Qifn6rG7-DNrg,4.0
...,...,...,...
967542,bJ5FtCtZX3ZZacz2_2PJjA,wMQkdK2aNMvq2xoojC98Mw,4.0
967543,bJ5FtCtZX3ZZacz2_2PJjA,SOsjW1JARmtHUFtpFlp8rw,4.0
967545,LHWtjTG7e1NzNPYUbUo-9w,rgeuy1qbw6Z8B6CSVANHIA,5.0
967550,7ziWZULyiZv2TesYNMFf4g,qQO7ErS_RAN4Vs1uX0L55Q,4.0


In [9]:
with open('yelp_academic_dataset_user.json', 'r') as f:
    data = [json.loads(line) for line in f]
df_frnds = pd.DataFrame(data)

In [10]:
df_frnds

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,qVc8ODYU5SZjKXVBgXdI7w,Walker,585,2007-01-25 16:47:26,7217,1259,5994,2007,"NSCy54eWehBJyZdG2iE84w, pe42u7DcCH2QmI81NX-8qA...",267,...,65,55,56,18,232,844,467,467,239,180
1,j14WgRoU_-2ZE1aw1dXrJg,Daniel,4333,2009-01-25 04:35:42,43091,13066,27281,"2009,2010,2011,2012,2013,2014,2015,2016,2017,2...","ueRPE0CX75ePGMqOFVj6IQ, 52oH4DrRvzzl8wh5UXyU0A...",3138,...,264,184,157,251,1847,7054,3131,3131,1521,1946
2,2WnXYQFK0hXEoTxPtV2zvg,Steph,665,2008-07-25 10:41:00,2086,1010,1003,"2009,2010,2011,2012,2013","LuO3Bn4f3rlhyHIaNfTlnA, j9B4XdHUhDfTKVecyWQgyA...",52,...,13,10,17,3,66,96,119,119,35,18
3,SZDeASXq7o05mMNLshsdIA,Gwen,224,2005-11-29 04:38:33,512,330,299,"2009,2010,2011","enx1vVPnfdNUdPho6PH_wg, 4wOcvMLtU6a9Lslggq74Vg...",28,...,4,1,6,2,12,16,26,26,10,9
4,hA5lMy-EnncsH4JoR-hFGQ,Karen,79,2007-01-05 19:40:59,29,15,7,,"PBK4q9KEEBHhFvSXCUirIw, 3FWPpM7KU1gXeOM_ZbYMbA...",1,...,1,0,0,0,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1987892,fB3jbHi3m0L2KgGOxBv6uw,Jerrold,23,2015-01-06 00:31:31,7,0,0,,None,0,...,0,0,0,0,0,0,0,0,0,0
1987893,68czcr4BxJyMQ9cJBm6C7Q,Jane,1,2016-06-14 07:20:52,0,0,0,,None,0,...,0,0,0,0,0,0,0,0,0,0
1987894,1x3KMskYxOuJCjRz70xOqQ,Shomari,4,2017-02-04 15:31:58,1,1,0,,None,0,...,0,0,0,0,0,0,0,0,0,0
1987895,ulfGl4tdbrH05xKzh5lnog,Susanne,2,2011-01-14 00:29:08,0,0,0,,None,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
#edit the friends collumn 
users = df_frnds[['user_id', 'friends']].copy()
users['friends'] = users['friends'].apply(lambda x: [friend.strip() for friend in x.split(',')] if x else [])

In [12]:
#keep only the friends that count 
user_ids = df_pruned["user_id"].unique().tolist()
users_friends = users[users['user_id'].isin(user_ids)]
users_friends["friends"] = users_friends["friends"].apply(lambda x: [i for i in x if i in user_ids])

/var/folders/gw/2h0k1fwd3x1cfqdc9_7xxxt80000gn/T/ipykernel_2747/1793770944.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  users_friends["friends"] = users_friends["friends"].apply(lambda x: [i for i in x if i in user_ids])


In [156]:
G = nx.Graph()
G.add_nodes_from(users_friends['user_id'])
G.number_of_nodes()

1749

In [14]:
for user_id, friends in zip(users_friends['user_id'], users_friends['friends']):
    for friend_id in friends:
        G.add_edge(user_id, friend_id)

In [15]:
largest_cc = max(nx.connected_components(G), key=len)
G.number_of_nodes()

1749

In [16]:
#largest connected components Graph
G_lcc = G.subgraph(largest_cc)

In [17]:
print(f"Number of nodes in LCC: {G_lcc.number_of_nodes()}")
print(f"Number of edges in LCC: {G_lcc.number_of_edges()}")

Number of nodes in LCC: 1630
Number of edges in LCC: 20578


In [18]:
edges = list(G_lcc.nodes())


In [19]:
#final Users and Businesses df
df_pruned = df_pruned[df_pruned['user_id'].isin(edges)]
df_pruned

,user_id,business_id,stars
3,smOvOajNG0lS4Pq7d8g4JQ,RZtGWDLCAtuipwaZ-UfjmQ,4.0
5,IQsF3Rc6IgCzjVV9DE8KXg,eFvzHawVJofxSnD7TgbZtg,5.0
10,ZGjgfSvjQK886kiTzLwfLQ,EtKSTHV5Qx_Q7Aur9o4kQQ,5.0
11,IKbjLnfBQtEyVzEu8CuOLg,VJEzpfLs_Jnzgqh5A_FVTg,4.0
13,NUtIAX-ygn474tDg5nmesg,6LCZLGa09Qifn6rG7-DNrg,4.0
...,...,...,...
967542,bJ5FtCtZX3ZZacz2_2PJjA,wMQkdK2aNMvq2xoojC98Mw,4.0
967543,bJ5FtCtZX3ZZacz2_2PJjA,SOsjW1JARmtHUFtpFlp8rw,4.0
967545,LHWtjTG7e1NzNPYUbUo-9w,rgeuy1qbw6Z8B6CSVANHIA,5.0
967550,7ziWZULyiZv2TesYNMFf4g,qQO7ErS_RAN4Vs1uX0L55Q,4.0


In [20]:
U = df_pruned['user_id'].unique().tolist()
len(U)

1630

In [21]:
B = df_pruned['business_id'].unique().tolist()
len(B)

940

                                                - Step 2 -

In [22]:
import random

random_user_ids = random.sample(U, k=100)
random_business_ids = random.sample(B, k=100)
df_test = df_pruned[df_pruned['user_id'].isin(random_user_ids) & df_pruned['business_id'].isin(random_business_ids)]

In [23]:
df_test

,user_id,business_id,stars
1566,Qr4WZ5cQ1HtCowUy27BdUQ,GUMVStVaJfWmCIpQA4TtHg,3.0
2406,7He2tF83YGi8cgbPXByEUg,EtKSTHV5Qx_Q7Aur9o4kQQ,4.0
5241,KSrQ-qr-dXCcJadevuS1RA,EtKSTHV5Qx_Q7Aur9o4kQQ,3.0
5580,dIRTUvNn9HSG7SlCB1CdKw,-ATiAtTikuGuqvaW2O6tNA,2.0
7128,quBsSLsr7J5XiA7mydCvcg,EtKSTHV5Qx_Q7Aur9o4kQQ,2.0
...,...,...,...
957601,jxac2pQOPQvYY32Zk-SM1Q,0RuvlgTnKFbX3IK0ZOOocA,1.0
960256,Vjb8dsOwTXQU-4bdmNzfbw,0RuvlgTnKFbX3IK0ZOOocA,3.0
965373,6EWf9mJbX7sz0hQTPlbfHw,woTZpX92uByUgliPrh39Mg,5.0
965517,u1P7POW_Hy4V8xaOWbvf8g,woTZpX92uByUgliPrh39Mg,4.0


In [26]:
test_rows = [tuple(x) for x in df_test[['user_id', 'business_id', 'stars']].values]
df_train = df_pruned[~df_pruned[['user_id', 'business_id', 'stars']].apply(tuple, axis=1).isin(test_rows)]

df_train

,user_id,business_id,stars
3,smOvOajNG0lS4Pq7d8g4JQ,RZtGWDLCAtuipwaZ-UfjmQ,4.0
5,IQsF3Rc6IgCzjVV9DE8KXg,eFvzHawVJofxSnD7TgbZtg,5.0
10,ZGjgfSvjQK886kiTzLwfLQ,EtKSTHV5Qx_Q7Aur9o4kQQ,5.0
11,IKbjLnfBQtEyVzEu8CuOLg,VJEzpfLs_Jnzgqh5A_FVTg,4.0
13,NUtIAX-ygn474tDg5nmesg,6LCZLGa09Qifn6rG7-DNrg,4.0
...,...,...,...
967542,bJ5FtCtZX3ZZacz2_2PJjA,wMQkdK2aNMvq2xoojC98Mw,4.0
967543,bJ5FtCtZX3ZZacz2_2PJjA,SOsjW1JARmtHUFtpFlp8rw,4.0
967545,LHWtjTG7e1NzNPYUbUo-9w,rgeuy1qbw6Z8B6CSVANHIA,5.0
967550,7ziWZULyiZv2TesYNMFf4g,qQO7ErS_RAN4Vs1uX0L55Q,4.0


                                                 -Step 3-

In [396]:
df_test = pd.read_csv('test_data.csv')
df_test

,Unnamed: 0,user_id,business_id,stars
0,4267,mzL0zHFGyd9866OUljq49g,7lwe7n-Yc-V9E_HfLAeylg,3.0
1,8216,5GJ5aLTtTjGIiBz7i9GssQ,4_W5pstoN1166TGjjPOrMg,5.0
2,8611,-dnERQYQCDVZFMMuh7bBDg,XQWZQjC73fCTRGWnD18-Eg,3.0
3,11696,PV0U0arT_AUfjVBtrKfPSA,AlxnbBd2JNkozNvI0OvRLQ,5.0
4,12166,uxOjXmWuuEuHmF6rXFZfSw,4_W5pstoN1166TGjjPOrMg,4.0
...,...,...,...,...
792,963134,ET8n-r7glWYqZhuR6GcdNw,hHs2Ytn2QSHnFscnALsLlQ,4.0
793,963998,ET8n-r7glWYqZhuR6GcdNw,el-WV1mOpFmrQOowLnh4aw,5.0
794,964207,dioXbYFdMCyE7zCK90eM3Q,1OfhM-ZKvcpxyxptCCzEwA,4.0
795,965001,zfD3xhVNkGJs-AOOSslqtQ,1OfhM-ZKvcpxyxptCCzEwA,3.0


In [397]:
df_train = pd.read_csv('train_data.csv')
df_train

,Unnamed: 0,user_id,business_id,stars
0,3,smOvOajNG0lS4Pq7d8g4JQ,RZtGWDLCAtuipwaZ-UfjmQ,4.0
1,5,IQsF3Rc6IgCzjVV9DE8KXg,eFvzHawVJofxSnD7TgbZtg,5.0
2,10,ZGjgfSvjQK886kiTzLwfLQ,EtKSTHV5Qx_Q7Aur9o4kQQ,5.0
3,11,IKbjLnfBQtEyVzEu8CuOLg,VJEzpfLs_Jnzgqh5A_FVTg,4.0
4,13,NUtIAX-ygn474tDg5nmesg,6LCZLGa09Qifn6rG7-DNrg,4.0
...,...,...,...,...
96589,967542,bJ5FtCtZX3ZZacz2_2PJjA,wMQkdK2aNMvq2xoojC98Mw,4.0
96590,967543,bJ5FtCtZX3ZZacz2_2PJjA,SOsjW1JARmtHUFtpFlp8rw,4.0
96591,967545,LHWtjTG7e1NzNPYUbUo-9w,rgeuy1qbw6Z8B6CSVANHIA,5.0
96592,967550,7ziWZULyiZv2TesYNMFf4g,qQO7ErS_RAN4Vs1uX0L55Q,4.0


In [27]:
#wanna recreate the graph from step1 to be in part with the given database (since mine is slightly different)
df_G = pd.concat([df_train, df_test], axis=0)
user_ids_G = df_G["user_id"].unique().tolist()
users_frds = users[users['user_id'].isin(user_ids_G)]
users_frds["friends"] = users_friends["friends"].apply(lambda x: [i for i in x if i in user_ids_G])

/var/folders/gw/2h0k1fwd3x1cfqdc9_7xxxt80000gn/T/ipykernel_2747/4293480144.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  users_frds["friends"] = users_friends["friends"].apply(lambda x: [i for i in x if i in user_ids_G])


In [28]:
G1 = nx.Graph()
G1.add_nodes_from(users_frds['user_id'])
for user_id, friends in zip(users_frds['user_id'], users_frds['friends']):
    for friend_id in friends:
        G1.add_edge(user_id, friend_id)

In [29]:
Btest = df_test['business_id'].unique()

In [30]:
ratings = {(row['user_id'], row['business_id']): row['stars'] for index, row in df_train.iterrows()}
 # dictionary ratings

In [155]:
predictions = {}
threshold = 10**(-6)

# get the set of unique business ids in df_test
for b in Btest:
    #print(b)
    users_rated_b = set(df_train[df_train['business_id'] == b]['user_id'])
    users_not_rated_b = set(df_test[df_test['business_id'] == b]['user_id']).difference(users_rated_b)
    
    
    node_values = {node: 0 for node in G1.nodes} #inizialization

    # set the node values for the users who rated business b
    for i, row in df_train[df_train['business_id'] == b].iterrows():
        user = row['user_id']
        node_values[user] = row['stars']

    # initialize the next node values dictionary for the value propagation
    next_node_values = {node: 0 for node in G1.nodes}
    for node in G1.nodes:
        if node in users_rated_b: # for every set of 3 (u,b,r) constant value r on node
            continue
        else: 
            neighbors = list(G1.neighbors(node))
            neighbors_list = []
            for n in neighbors:
                if n in node_values and n in users_rated_b :#get rating for users tha have rated b
                    neighbors_list += [n]
            if len(neighbors_list) != 0:
                neighbor_values = [node_values[neighbor] for neighbor in neighbors_list]
                next_node_values[node] = sum(neighbor_values) / len(neighbors_list)
            else:
                next_node_values[node] = 0

            while True: #Repetive 
                new_values = []
                new_values.append(next_node_values[node])
                for neighbor in neighbors_list: #get the neighbors' neighbors rating
                    if neighbor in node_values:
                        new_values.append(node_values[neighbor])
                    else:
                        new_values.append(next_node_values[neighbor])
                if len(new_values) != 0:
                    node_values[node] = sum(new_values) / len(new_values)
                else:
                    node_values[node] = 0
                diff = abs(node_values[node] - next_node_values[node])
                if diff < threshold: #repeat until threshold
                    break
                next_node_values[node] = node_values[node]

        predictions.update({(node, b): node_values[node]})

In [154]:
#RMSE calculation
from sklearn.metrics import mean_squared_error
import math
stars = []
predicted = []
for index, row in df_test.iterrows():
    stars.append(row['stars'])
    predicted.append(predictions[(row['user_id'], row['business_id'])])

mse = mean_squared_error(stars, predicted)
rmse = math.sqrt(mse)
print("RMSE:", rmse)

RMSE: 3.686079702332026


                                                -Step 4-

In [398]:
from sklearn.metrics.pairwise import cosine_similarity

In [399]:
df_test

,Unnamed: 0,user_id,business_id,stars
0,4267,mzL0zHFGyd9866OUljq49g,7lwe7n-Yc-V9E_HfLAeylg,3.0
1,8216,5GJ5aLTtTjGIiBz7i9GssQ,4_W5pstoN1166TGjjPOrMg,5.0
2,8611,-dnERQYQCDVZFMMuh7bBDg,XQWZQjC73fCTRGWnD18-Eg,3.0
3,11696,PV0U0arT_AUfjVBtrKfPSA,AlxnbBd2JNkozNvI0OvRLQ,5.0
4,12166,uxOjXmWuuEuHmF6rXFZfSw,4_W5pstoN1166TGjjPOrMg,4.0
...,...,...,...,...
792,963134,ET8n-r7glWYqZhuR6GcdNw,hHs2Ytn2QSHnFscnALsLlQ,4.0
793,963998,ET8n-r7glWYqZhuR6GcdNw,el-WV1mOpFmrQOowLnh4aw,5.0
794,964207,dioXbYFdMCyE7zCK90eM3Q,1OfhM-ZKvcpxyxptCCzEwA,4.0
795,965001,zfD3xhVNkGJs-AOOSslqtQ,1OfhM-ZKvcpxyxptCCzEwA,3.0


In [400]:
from sklearn.preprocessing import OrdinalEncoder
df_G = pd.concat([df_train, df_test], axis=0)


from sklearn.preprocessing import OrdinalEncoder
ord_enc = OrdinalEncoder()
df_G["user_id_code"] = ord_enc.fit_transform(df_G[["user_id"]])
ord_enc1 =OrdinalEncoder()
df_G["business_id_code"] = ord_enc1.fit_transform(df_G[["business_id"]])
df_G

,Unnamed: 0,user_id,business_id,stars,user_id_code,business_id_code
0,3,smOvOajNG0lS4Pq7d8g4JQ,RZtGWDLCAtuipwaZ-UfjmQ,4.0,1329.0,396.0
1,5,IQsF3Rc6IgCzjVV9DE8KXg,eFvzHawVJofxSnD7TgbZtg,5.0,454.0,587.0
2,10,ZGjgfSvjQK886kiTzLwfLQ,EtKSTHV5Qx_Q7Aur9o4kQQ,5.0,881.0,220.0
3,11,IKbjLnfBQtEyVzEu8CuOLg,VJEzpfLs_Jnzgqh5A_FVTg,4.0,450.0,442.0
4,13,NUtIAX-ygn474tDg5nmesg,6LCZLGa09Qifn6rG7-DNrg,4.0,562.0,108.0
...,...,...,...,...,...,...
792,963134,ET8n-r7glWYqZhuR6GcdNw,hHs2Ytn2QSHnFscnALsLlQ,4.0,369.0,636.0
793,963998,ET8n-r7glWYqZhuR6GcdNw,el-WV1mOpFmrQOowLnh4aw,5.0,369.0,599.0
794,964207,dioXbYFdMCyE7zCK90eM3Q,1OfhM-ZKvcpxyxptCCzEwA,4.0,1000.0,44.0
795,965001,zfD3xhVNkGJs-AOOSslqtQ,1OfhM-ZKvcpxyxptCCzEwA,3.0,1495.0,44.0


In [402]:
df_test["user_id_code"] = ord_enc.fit_transform(df_test[["user_id"]])
df_test["business_id_code"] = ord_enc1.fit_transform(df_test[["business_id"]])
df_train["user_id_code"] = ord_enc.fit_transform(df_train[["user_id"]])
df_train["business_id_code"] = ord_enc1.fit_transform(df_train[["business_id"]])
#df_test['user_id_code'] = df_G['user_id_code'].astype(int)
#df_test['business_id_code'] = df_G['business_id_code'].astype(int)
#df_train['user_id_code'] = df_G['user_id_code'].astype(int)
#df_train['business_id_code'] = df_G['business_id_code'].astype(int)
user_avg = df_G.groupby('user_id')['stars'].mean()
df_G['user_avg'] = df_G['user_id'].map(user_avg)
df_G['stars_s5'] = df_G['stars'] - df_G['user_avg']


In [403]:
from scipy.sparse import csr_matrix
df_G['user_id_code'] = df_G['user_id_code'].astype(int)
df_G['business_id_code'] = df_G['business_id_code'].astype(int)
r_s5 = csr_matrix((df_G['stars_s5'].values, (df_G['user_id_code'].values, df_G['business_id_code'].values)))

In [404]:
def get_user_avg(user_id_code,df_s3):
    business_df = df_s3[df_s3['user_id_code'] == user_id_code]
    return business_df['stars'].mean()

In [434]:
import bisect
def p_value(user_id,business_id,k):
    counter = 0
    #Get the businesses (step1):
    users, businesses = r_s5.nonzero()
    users_of_interest = users[businesses == business_id]
    #some user_id are not in here even tho they are in df_G
    if user_id not in users_of_interest:
        index = [bisect.bisect_left(users_of_interest, user_id)]
    else:
        index = np.where(users_of_interest == user_id)[0]
    #users_of_interest = np.delete(users_of_interest, index)
    #Get most similar (step 2):
    if(k >= len(users_of_interest)):
        most_similar_users = users_of_interest
        most_similar_users = np.delete(users_of_interest, index)
    elif(k < len(users_of_interest)):
        user_ratings = r_s5[users_of_interest, :].toarray()
        user_similarity = cosine_similarity(user_ratings)
        most_similar_indices = np.argsort(-user_similarity[index[0]])[:k+1]
        most_similar_indices = most_similar_indices[1:]
        most_similar_users = [users_of_interest[i] for i in most_similar_indices]
    else: #incase of zero values
        return 0
    #Get similarity scores
    user_vector = r_s5[user_id,:].toarray() 
    similar_users_vectors = r_s5[most_similar_users,:].toarray()
    similarity_scores = cosine_similarity(user_vector, similar_users_vectors)
    similarity_scores = similarity_scores.flatten() 
    #R[u',b]
    stars_s5 = r_s5[most_similar_users, business_id].toarray().flatten()
    return get_user_avg(user_id,df_train)+ np.sum(stars_s5*similarity_scores)/np.sum(similarity_scores)

In [406]:
df_test

,Unnamed: 0,user_id,business_id,stars,user_id_code,business_id_code
0,4267,mzL0zHFGyd9866OUljq49g,7lwe7n-Yc-V9E_HfLAeylg,3.0,77.0,14.0
1,8216,5GJ5aLTtTjGIiBz7i9GssQ,4_W5pstoN1166TGjjPOrMg,5.0,5.0,8.0
2,8611,-dnERQYQCDVZFMMuh7bBDg,XQWZQjC73fCTRGWnD18-Eg,3.0,1.0,51.0
3,11696,PV0U0arT_AUfjVBtrKfPSA,AlxnbBd2JNkozNvI0OvRLQ,5.0,34.0,19.0
4,12166,uxOjXmWuuEuHmF6rXFZfSw,4_W5pstoN1166TGjjPOrMg,4.0,89.0,8.0
...,...,...,...,...,...,...
792,963134,ET8n-r7glWYqZhuR6GcdNw,hHs2Ytn2QSHnFscnALsLlQ,4.0,22.0,71.0
793,963998,ET8n-r7glWYqZhuR6GcdNw,el-WV1mOpFmrQOowLnh4aw,5.0,22.0,67.0
794,964207,dioXbYFdMCyE7zCK90eM3Q,1OfhM-ZKvcpxyxptCCzEwA,4.0,62.0,4.0
795,965001,zfD3xhVNkGJs-AOOSslqtQ,1OfhM-ZKvcpxyxptCCzEwA,3.0,98.0,4.0


In [435]:
df_test['ucf_predicted'] = 0.0

for i, row in df_test.iterrows():
    # Get the user_id and business_id
    user_id = row['user_id_code']
    business_id = row['business_id_code']
   
    predicted_rating = p_value(user_id, business_id, 100)
    if predicted_rating > 5:
        predicted_rating=5
    if predicted_rating < 0:
        predicted_rating=0
    df_test.loc[i, 'ucf_predicted'] = predicted_rating

In [436]:
df_test

,Unnamed: 0,user_id,business_id,stars,user_id_code,business_id_code,ucf_predicted,icf_predicted
0,4267,mzL0zHFGyd9866OUljq49g,7lwe7n-Yc-V9E_HfLAeylg,3.0,77.0,14.0,3.887802,5.0
1,8216,5GJ5aLTtTjGIiBz7i9GssQ,4_W5pstoN1166TGjjPOrMg,5.0,5.0,8.0,4.065831,5.0
2,8611,-dnERQYQCDVZFMMuh7bBDg,XQWZQjC73fCTRGWnD18-Eg,3.0,1.0,51.0,4.120101,5.0
3,11696,PV0U0arT_AUfjVBtrKfPSA,AlxnbBd2JNkozNvI0OvRLQ,5.0,34.0,19.0,5.000000,5.0
4,12166,uxOjXmWuuEuHmF6rXFZfSw,4_W5pstoN1166TGjjPOrMg,4.0,89.0,8.0,4.622190,5.0
...,...,...,...,...,...,...,...,...
792,963134,ET8n-r7glWYqZhuR6GcdNw,hHs2Ytn2QSHnFscnALsLlQ,4.0,22.0,71.0,3.755899,5.0
793,963998,ET8n-r7glWYqZhuR6GcdNw,el-WV1mOpFmrQOowLnh4aw,5.0,22.0,67.0,3.732545,5.0
794,964207,dioXbYFdMCyE7zCK90eM3Q,1OfhM-ZKvcpxyxptCCzEwA,4.0,62.0,4.0,3.663528,5.0
795,965001,zfD3xhVNkGJs-AOOSslqtQ,1OfhM-ZKvcpxyxptCCzEwA,3.0,98.0,4.0,3.724434,5.0


In [437]:
#RMSE calculation
actual_ratings = df_test['stars'].values
predicted_ratings = df_test['ucf_predicted'].values

# Calculate the RMSE
rmse = np.sqrt(mean_squared_error(actual_ratings, predicted_ratings))
print("UCF RMSE:", rmse)

UCF RMSE: 1.1570455085538367


In [410]:
matrix = csr_matrix((df_G['stars'].values, (df_G['user_id_code'].values, df_G['business_id_code'].values)))

In [429]:
def p_val_icf(u,b,k):
#Step1: find the businesses that u has rated
    users, businesses = matrix.nonzero()
    businesses_of_interest = businesses[users == u]
    #users_of_interest = users[businesses == b]
    #Step 2:
    index = np.where(businesses_of_interest == b)[0]
    business_ratings = matrix[:,businesses_of_interest].toarray().T
    business_vector = matrix[:,b].toarray().T
    business_similarity = cosine_similarity(business_ratings,business_vector).T
    most_similar_indices = np.argsort(business_similarity[0])
    most_similar_indices = most_similar_indices[-k-1:-1]
    most_similar_businesses = [businesses_of_interest[i] for i in most_similar_indices]
    most_similar_businesses_scores = [business_similarity[0][i] for i in most_similar_indices]
    #find similarities
    other_avg = matrix[u,most_similar_businesses].toarray().flatten()
    #find similarities
    #compute p(u,b)
    for i in range(len(other_avg)):
        if (other_avg[i]>5):
            other_avg[i]=5
        elif(other_avg[i]<0):
            other_avg[i]=5
    #compute p(u,b)
    return get_user_avg(u,df_train)+ np.sum(other_avg*most_similar_businesses_scores)/np.sum(most_similar_businesses_scores)
#this is more simple that the function for the UCF and also kinda faster (uses only one time the cosin.similarity
#and itarates less times the big sparce matrix 

In [426]:
df_test['icf_predicted'] = 0.0

for i, row in df_test.iterrows():
    # Get the user_id and business_id
    user_id = row['user_id_code']
    business_id = row['business_id_code']
   
    predicted_rating = p_val_icf(user_id, business_id, 100)
    if predicted_rating > 5:
        predicted_rating=5
    if predicted_rating < 0:
        predicted_rating=0
    df_test.loc[i, 'icf_predicted'] = predicted_rating

In [427]:
#RMSE calculation
actual_ratings = df_test['stars'].values
predicted_ratings = df_test['icf_predicted'].values

# Calculate the RMSE
rmse = np.sqrt(mean_squared_error(actual_ratings, predicted_ratings))
print("ICF RMSE:", rmse)

ICF RMSE: 1.4309712769594858


In [443]:
user_averages = df_train.groupby('user_id')['stars'].mean()
def predict_user_average(user_id):
    return user_averages[user_id]
df_test["ua_predicted"] = df_test["user_id"].apply(predict_user_average)

In [445]:
#RMSE calculation
actual_ratings = df_test['stars'].values
predicted_ratings = df_test['ua_predicted'].values

# Calculate the RMSE
rmse = np.sqrt(mean_squared_error(actual_ratings, predicted_ratings))
print("UA RMSE:", rmse)

UA RMSE: 0.9012981678615509


The accuracy of the value propagation  prediction algorithm can be improved, maybe its wrong to use the ratings of the neighbors' neighbor in the graph.Maybe sth more sequential. Since i noticed tha most ratings tended to 3.7 maybe i shoud't give the save weight to all the interations as the last interation.
UA which is a simple model scored the highest.Then the UCF model is a user-based collaborative filtering model, where the similarity between users is calculated based on their ratings.Lastly the item-based ICF mode